In [104]:
# import packages/modules
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import config
from sodapy import Socrata
import mysql.connector
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols

In [28]:
# create connection
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'nyc_public_high_schools_grad_rate',
    use_pure=True
    )

cursor = cnx.cursor()

In [54]:
pd.set_option('display.max_rows', 100)

In [32]:
regents = pd.read_csv('regentsfinal.csv')
regents = regents.drop(columns=['Unnamed: 0'])
regents

,school_name,regents_exam,percent_scoring_65_or_above
0,Orchard Collegiate Academy,Common Core Algebra,71.7
1,Orchard Collegiate Academy,Common Core Algebra2,NaN
2,Orchard Collegiate Academy,Common Core English,92.7
3,Orchard Collegiate Academy,Common Core Geometry,18.2
4,Orchard Collegiate Academy,Global History and Geography,39.6
...,...,...,...
21109,KIPP Academy Charter School,Living Environment,87.7
21110,KIPP Academy Charter School,Physical Settings/Chemistry,79.0
21111,KIPP Academy Charter School,Physical Settings/Earth Science,36.0
21112,KIPP Academy Charter School,Physical Settings/Physics,55.3


In [38]:
algebra = regents[regents['regents_exam']=='Common Core Algebra']
algebra

,school_name,regents_exam,percent_scoring_65_or_above
0,Orchard Collegiate Academy,Common Core Algebra,71.7
9,University Neighborhood High School,Common Core Algebra,80.4
19,East Side Community School,Common Core Algebra,83.3
23,Forsyth Satellite Academy,Common Core Algebra,0.0
29,Marta Valle High School,Common Core Algebra,39.5
...,...,...,...
21071,New Visions Charter High School for Advanced Math,Common Core Algebra,68.1
21082,New Visions Charter High School for the Humani...,Common Core Algebra,63.8
21092,South Bronx Community Charter School,Common Core Algebra,75.0
21094,Bronx Preparatory Charter School,Common Core Algebra,88.7


In [39]:
algebra2 = regents[regents['regents_exam']=='Common Core Algebra2']
algebra2

,school_name,regents_exam,percent_scoring_65_or_above
1,Orchard Collegiate Academy,Common Core Algebra2,NaN
10,University Neighborhood High School,Common Core Algebra2,61.0
30,Marta Valle High School,Common Core Algebra2,16.7
40,Lower East Side Preparatory High School,Common Core Algebra2,82.1
48,"New Explorations into Science, Technology and ...",Common Core Algebra2,94.9
...,...,...,...
21062,The Equality Charter School,Common Core Algebra2,64.9
21072,New Visions Charter High School for Advanced Math,Common Core Algebra2,34.5
21083,New Visions Charter High School for the Humani...,Common Core Algebra2,46.7
21095,Bronx Preparatory Charter School,Common Core Algebra2,25.7


In [41]:
english = regents[regents['regents_exam']=='Common Core English']
english

,school_name,regents_exam,percent_scoring_65_or_above
2,Orchard Collegiate Academy,Common Core English,92.7
11,University Neighborhood High School,Common Core English,82.0
20,East Side Community School,Common Core English,92.7
24,Forsyth Satellite Academy,Common Core English,65.8
31,Marta Valle High School,Common Core English,69.6
...,...,...,...
21063,The Equality Charter School,Common Core English,71.3
21073,New Visions Charter High School for Advanced Math,Common Core English,59.0
21084,New Visions Charter High School for the Humani...,Common Core English,79.6
21096,Bronx Preparatory Charter School,Common Core English,87.4


In [42]:
geometry = regents[regents['regents_exam']=='Common Core Geometry']
geometry

,school_name,regents_exam,percent_scoring_65_or_above
3,Orchard Collegiate Academy,Common Core Geometry,18.2
12,University Neighborhood High School,Common Core Geometry,68.6
21,East Side Community School,Common Core Geometry,NaN
32,Marta Valle High School,Common Core Geometry,53.3
42,Lower East Side Preparatory High School,Common Core Geometry,79.0
...,...,...,...
21064,The Equality Charter School,Common Core Geometry,47.2
21074,New Visions Charter High School for Advanced Math,Common Core Geometry,32.0
21085,New Visions Charter High School for the Humani...,Common Core Geometry,35.2
21097,Bronx Preparatory Charter School,Common Core Geometry,67.0


In [46]:
globalgeo = regents[regents['regents_exam']=='Global History and Geography']
globalgeo

,school_name,regents_exam,percent_scoring_65_or_above
4,Orchard Collegiate Academy,Global History and Geography,39.6
13,University Neighborhood High School,Global History and Geography,71.0
25,Forsyth Satellite Academy,Global History and Geography,12.5
33,Marta Valle High School,Global History and Geography,34.9
43,Lower East Side Preparatory High School,Global History and Geography,67.9
...,...,...,...
21065,The Equality Charter School,Global History and Geography,46.9
21075,New Visions Charter High School for Advanced Math,Global History and Geography,44.0
21086,New Visions Charter High School for the Humani...,Global History and Geography,46.3
21098,Bronx Preparatory Charter School,Global History and Geography,68.9


In [44]:
livenv = regents[regents['regents_exam']=='Living Environment']
livenv

,school_name,regents_exam,percent_scoring_65_or_above
5,Orchard Collegiate Academy,Living Environment,57.4
14,University Neighborhood High School,Living Environment,80.9
22,East Side Community School,Living Environment,NaN
26,Forsyth Satellite Academy,Living Environment,0.0
34,Marta Valle High School,Living Environment,32.7
...,...,...,...
21076,New Visions Charter High School for Advanced Math,Living Environment,62.6
21087,New Visions Charter High School for the Humani...,Living Environment,51.8
21093,South Bronx Community Charter School,Living Environment,52.6
21099,Bronx Preparatory Charter School,Living Environment,83.3


In [45]:
earthsci = regents[regents['regents_exam']=='Physical Settings/Earth Science']
earthsci

,school_name,regents_exam,percent_scoring_65_or_above
6,Orchard Collegiate Academy,Physical Settings/Earth Science,27.8
16,University Neighborhood High School,Physical Settings/Earth Science,75.8
27,Forsyth Satellite Academy,Physical Settings/Earth Science,NaN
36,Marta Valle High School,Physical Settings/Earth Science,0.0
45,Lower East Side Preparatory High School,Physical Settings/Earth Science,95.3
...,...,...,...
21058,Dr. Richard Izquierdo Health and Science Chart...,Physical Settings/Earth Science,80.0
21068,The Equality Charter School,Physical Settings/Earth Science,37.0
21078,New Visions Charter High School for Advanced Math,Physical Settings/Earth Science,44.9
21089,New Visions Charter High School for the Humani...,Physical Settings/Earth Science,53.1


In [47]:
histgov = regents[regents['regents_exam']=='U.S. History and Government']
histgov

,school_name,regents_exam,percent_scoring_65_or_above
7,Orchard Collegiate Academy,U.S. History and Government,76.2
18,University Neighborhood High School,U.S. History and Government,80.0
28,Forsyth Satellite Academy,U.S. History and Government,16.7
37,Marta Valle High School,U.S. History and Government,40.4
46,Lower East Side Preparatory High School,U.S. History and Government,89.0
...,...,...,...
21070,The Equality Charter School,U.S. History and Government,80.0
21080,New Visions Charter High School for Advanced Math,U.S. History and Government,56.2
21091,New Visions Charter High School for the Humani...,U.S. History and Government,81.7
21102,Bronx Preparatory Charter School,U.S. History and Government,73.4


In [51]:
chem = regents[regents['regents_exam']=='Physical Settings/Chemistry']
chem

,school_name,regents_exam,percent_scoring_65_or_above
15,University Neighborhood High School,Physical Settings/Chemistry,42.9
35,Marta Valle High School,Physical Settings/Chemistry,13.9
53,"New Explorations into Science, Technology and ...",Physical Settings/Chemistry,94.0
69,Bard High School Early College,Physical Settings/Chemistry,94.3
103,M.S. 260 Clinton School Writers & Artists,Physical Settings/Chemistry,NaN
...,...,...,...
21067,The Equality Charter School,Physical Settings/Chemistry,14.3
21077,New Visions Charter High School for Advanced Math,Physical Settings/Chemistry,15.6
21088,New Visions Charter High School for the Humani...,Physical Settings/Chemistry,14.8
21100,Bronx Preparatory Charter School,Physical Settings/Chemistry,52.7


In [52]:
physics = regents[regents['regents_exam']=='Physical Settings/Physics']
physics

,school_name,regents_exam,percent_scoring_65_or_above
17,University Neighborhood High School,Physical Settings/Physics,NaN
55,"New Explorations into Science, Technology and ...",Physical Settings/Physics,83.5
104,M.S. 260 Clinton School Writers & Artists,Physical Settings/Physics,NaN
154,Pace High School,Physical Settings/Physics,11.1
163,"Urban Assembly School of Design and Construction,",Physical Settings/Physics,NaN
164,Gramercy Arts High School,Physical Settings/Physics,NaN
255,The High School For Language And Diplomacy,Physical Settings/Physics,50.0
266,High School for Environmental Studies,Physical Settings/Physics,55.1
288,Baruch College Campus High School,Physical Settings/Physics,NaN
297,N.Y.C. Lab School for Collaborative Studies,Physical Settings/Physics,67.0


In [31]:
regents.regents_exam.unique()

array(['Common Core Algebra', 'Common Core Algebra2',
       'Common Core English', 'Common Core Geometry',
       'Global History and Geography', 'Living Environment',
       'Physical Settings/Earth Science', 'U.S. History and Government',
       'Algebra2/Trigonometry', 'Physical Settings/Chemistry',
       'Physical Settings/Physics', 'Integrated Algebra', 'Geometry'],
      dtype=object)

In [35]:
grad = pd.read_csv('gradfinal.csv')
grad = grad.drop(columns=['Unnamed: 0', 'index'])
grad

,school_name,total_cohort,total_grads,percent_graduated
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,35.0,32.0,91.4
1,A PHILIP RANDOLPH CAMPUS HIGH SCHOOL,331.0,233.0,70.4
2,ABRAHAM LINCOLN HIGH SCHOOL,484.0,325.0,67.1
3,ACADEMY FOR CAREERS IN TELEVISION AND FILM,121.0,119.0,98.3
4,ACADEMY FOR COLLEGE PREPARATION AND CAREER EXP...,63.0,39.0,61.9
5,ACADEMY FOR CONSERVATION AND THE ENVIRONMENT,72.0,58.0,80.6
6,ACADEMY FOR ENVIRONMENTAL LEADERSHIP,96.0,65.0,67.7
7,ACADEMY FOR HEALTH CAREERS,81.0,64.0,79.0
8,ACADEMY FOR LANGUAGE AND TECHNOLOGY,76.0,73.0,96.1
9,ACADEMY FOR SCHOLARSHIP AND ENTREPRENEURSHIP,96.0,57.0,59.4


In [37]:
att = pd.read_csv('attendancefinal.csv')
att = att.drop(columns=['Unnamed: 0', 'index'])
att

,school_name,attendance
0,A PHILIP RANDOLPH CAMPUS HIGH SCHOOL,85
1,ABRAHAM LINCOLN HIGH SCHOOL,84
2,ACAD FOR CAREERS IN TELEVISION-FILM,94
3,ACAD FOR COLLEGE PREP AND CAREER EXP,82
4,ACAD FOR CONSERVATION & ENVIRONMENT,81
5,ACAD-SCHOLARSHIP & ENTREPRENEURSHIP,82
6,ACADEMY FOR EXCELLENCE-LEADERSHIP,81
7,ACADEMY FOR HEALTH CAREERS,83
8,ACADEMY FOR LANGUAGE AND TECHNOLOGY,89
9,ACADEMY FOR SOFTWARE ENGINEERING,87


In [61]:
att_grad = pd.merge(grad, att, on='school_name')
att_grad = att_grad.dropna()

In [63]:
att_grad

,school_name,total_cohort,total_grads,percent_graduated,attendance
0,A PHILIP RANDOLPH CAMPUS HIGH SCHOOL,331.0,233.0,70.4,85
1,ABRAHAM LINCOLN HIGH SCHOOL,484.0,325.0,67.1,84
2,ACADEMY FOR HEALTH CAREERS,81.0,64.0,79.0,83
3,ACADEMY FOR LANGUAGE AND TECHNOLOGY,76.0,73.0,96.1,89
4,ACADEMY FOR SOFTWARE ENGINEERING,108.0,90.0,83.3,87
...,...,...,...,...,...
196,WINGS ACADEMY,107.0,50.0,46.7,77
197,WOMEN'S ACADEMY OF EXCELLENCE,67.0,45.0,67.2,75
198,WORLD VIEW HIGH SCHOOL,70.0,60.0,85.7,86
199,YORK EARLY COLLEGE ACADEMY,81.0,79.0,97.5,96


In [117]:
all_regents_grad_rate = pd.read_csv('all_regents_grad_rate.csv')
all_regents_grad_rate = all_regents_grad_rate.drop(columns=['Unnamed: 0'])
all_regents_grad_rate.columns

Index(['school_name', 'Common Core Algebra2', 'percent_graduated',
       'Common Core Algebra', 'Common Core English', 'Common Core Geometry',
       'Global History and Geography', 'Biology', 'Earth Science',
       'U.S. History and Government', 'Chemistry', 'Physics'],
      dtype='object')

In [124]:
all_regents_grad_rate.columns = [col.replace(' ', '_') for col in all_regents_grad_rate.columns]
all_regents_grad_rate.columns = [col.replace('.', '_') for col in all_regents_grad_rate.columns]

In [125]:
outcome = "percent_graduated"
x_cols = ['Common_Core_Algebra2','Common_Core_Algebra', 'Common_Core_English', 'Common_Core_Geometry','Global_History_and_Geography', 'Biology', 'Earth_Science','U_S__History_and_Government', 'Chemistry', 'Physics']

In [126]:
all_regents_grad_rate.columns

Index(['school_name', 'Common_Core_Algebra2', 'percent_graduated',
       'Common_Core_Algebra', 'Common_Core_English', 'Common_Core_Geometry',
       'Global_History_and_Geography', 'Biology', 'Earth_Science',
       'U_S__History_and_Government', 'Chemistry', 'Physics'],
      dtype='object')

In [127]:
for col in x_cols:
    all_regents_grad_rate[col] = (all_regents_grad_rate[col] - all_regents_grad_rate[col].mean())/all_regents_grad_rate[col].std()
all_regents_grad_rate

,school_name,Common_Core_Algebra2,percent_graduated,Common_Core_Algebra,Common_Core_English,Common_Core_Geometry,Global_History_and_Geography,Biology,Earth_Science,U_S__History_and_Government,Chemistry,Physics
0,Pace High School,-0.315345,79.3,0.043991,0.890172,-1.171632,0.287480,-0.438979,-0.219325,0.969919,0.037497,-1.734607
1,The High School For Language And Diplomacy,-0.543894,80.0,-0.592768,-0.438661,-1.145830,-0.861933,-0.503288,-0.508308,-0.700304,-1.192347,-0.258544
2,High School for Environmental Studies,-0.777014,76.7,-0.780901,-0.166292,-0.470657,-0.917730,-0.914863,-0.525822,-0.364768,-1.169642,-0.065024
3,High School for Health Professions and Human S...,0.694840,87.9,0.760345,0.642563,0.595859,0.460450,0.532080,-0.954918,0.313760,0.915417,0.553481
4,Manhattan Village Academy,1.705025,96.6,1.672068,1.311107,1.744083,1.554067,1.696065,1.768528,1.327823,1.191659,1.638709
5,High School of Economics and Finance,-0.809011,86.1,0.174237,0.642563,-0.079314,-0.192372,0.107644,0.450591,0.597101,-1.332360,-1.355156
6,Urban Assembly Gateway School for Technology,-0.767872,82.7,0.413022,-0.397393,-0.535164,0.599942,0.969379,0.888444,0.157176,-0.795013,-0.892226
7,Manhattan Bridges High School,-0.169074,81.3,0.767581,-0.133277,0.294826,0.332118,0.570665,0.752709,0.425605,1.630619,-0.471036
8,High School for Dual Language and Asian Studies,0.932531,100.0,1.954268,1.236824,1.924703,1.615443,1.696065,1.877992,1.387474,1.267341,1.448984
9,The High School of Fashion Industries,-0.365626,89.1,0.340663,1.055244,-0.388948,0.265161,0.577096,0.669517,0.544907,0.442400,-0.505187


In [128]:
# Fitting the actual model
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=all_regents_grad_rate).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      percent_graduated   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.671
Method:                 Least Squares   F-statistic:                     18.78
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           6.94e-17
Time:                        15:22:29   Log-Likelihood:                -295.70
No. Observations:                  88   AIC:                             613.4
Df Residuals:                      77   BIC:                             640.7
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       80.5193      0.794    101.400      0.000      78.938      82.101
Common_Core_Algebra2            -0.5090      1.467     -0.347      0.730      -3.431       2.413
Common_Core_Algebra              3.7061      1.769      2.095      0.039       0.184       7.228
Common_Core_English              3.9518      1.212      3.260      0.002       1.538       6.366
Common_Core_Geometry             0.6219      1.501      0.414      0.680      -2.367       3.611
Global_History_and_Geography     2.3516      1.647      1.428      0.157      -0.927       5.630
Biology                          2.1244      1.797      1.182      0.241      -1.454       5.703
Earth_Science                   -0.8846      1.234     -0.717      0.475      -3.341       1.572
U_S__History_and_Government      2.9917      1.403      2.132      0.036       0.198       5.785
Chemistry                       -1.9299      1.314     -1.469      0.146      -4.546       0.686
Physics                         -1.4617      1.184     -1.234      0.221      -3.820       0.896
==============================================================================
Omnibus:                        8.796   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.012   Jarque-Bera (JB):               11.827
Skew:                          -0.440   Prob(JB):                      0.00270
Kurtosis:                       4.565   Cond. No.                         7.09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [131]:
# Fitting the actual model
outcome = "percent_graduated"
x_cols = ['Common_Core_Algebra', 'Common_Core_English', 'U_S__History_and_Government']
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=all_regents_grad_rate).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      percent_graduated   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     54.62
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           1.10e-19
Time:                        15:35:45   Log-Likelihood:                -302.43
No. Observations:                  88   AIC:                             612.9
Df Residuals:                      84   BIC:                             622.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      80.5193      0.821     98.111      0.000      78.887      82.151
Common_Core_Algebra             3.1944      1.188      2.689      0.009       0.832       5.556
Common_Core_English             4.7900      1.133      4.229      0.000       2.537       7.043
U_S__History_and_Government     4.0945      1.222      3.350      0.001       1.664       6.525
==============================================================================
Omnibus:                        2.340   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.310   Jarque-Bera (JB):                1.966
Skew:                          -0.021   Prob(JB):                        0.374
Kurtosis:                       3.731   Cond. No.                         2.70
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [94]:
# plt.scatter((att_grad['attendance']),(att_grad['percent_graduated']))
# plt.xlabel('attendance')
# plt.ylabel('Percent_graduated')
# print(plt.show())
x=sm.OLS(all_regents_grad_rate['percent_graduated'], all_regents_grad_rate['Common Core Algebra2'])
results = x.fit()
print(results.summary())
print(stats.pearsonr(list(all_regents_grad_rate['percent_graduated']),list(all_regents_grad_rate['Common Core Algebra2'])))

                                 OLS Regression Results                                
Dep. Variable:      percent_graduated   R-squared (uncentered):                   0.909
Model:                            OLS   Adj. R-squared (uncentered):              0.908
Method:                 Least Squares   F-statistic:                              864.8
Date:                Thu, 16 Jan 2020   Prob (F-statistic):                    5.67e-47
Time:                        14:53:12   Log-Likelihood:                         -406.91
No. Observations:                  88   AIC:                                      815.8
Df Residuals:                      87   BIC:                                      818.3
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [88]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(att_grad, att_grad['percent_graduated'], test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(158, 5) (158,)
(40, 5) (40,)


In [89]:
# fit a model
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

ValueError: could not convert string to float: 'PROFESSIONAL PATHWAYS HIGH SCHOOL'

In [19]:
grad = pd.read_csv('gradfinal.csv')
grad = grad.drop(columns=['Unnamed: 0', 'index'])
grad

,school_name,total_cohort,total_grads,percent_graduated
0,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,35.0,32.0,91.4
1,A PHILIP RANDOLPH CAMPUS HIGH SCHOOL,331.0,233.0,70.4
2,ABRAHAM LINCOLN HIGH SCHOOL,484.0,325.0,67.1
3,ACADEMY FOR CAREERS IN TELEVISION AND FILM,121.0,119.0,98.3
4,ACADEMY FOR COLLEGE PREPARATION AND CAREER EXP...,63.0,39.0,61.9
5,ACADEMY FOR CONSERVATION AND THE ENVIRONMENT,72.0,58.0,80.6
6,ACADEMY FOR ENVIRONMENTAL LEADERSHIP,96.0,65.0,67.7
7,ACADEMY FOR HEALTH CAREERS,81.0,64.0,79.0
8,ACADEMY FOR LANGUAGE AND TECHNOLOGY,76.0,73.0,96.1
9,ACADEMY FOR SCHOLARSHIP AND ENTREPRENEURSHIP,96.0,57.0,59.4


In [21]:
att = pd.read_csv('attendancefinal.csv')
att = att.drop(columns=['Unnamed: 0', 'index'])
att

,school_name,attendance
0,A PHILIP RANDOLPH CAMPUS HIGH SCHOOL,85
1,ABRAHAM LINCOLN HIGH SCHOOL,84
2,ACAD FOR CAREERS IN TELEVISION-FILM,94
3,ACAD FOR COLLEGE PREP AND CAREER EXP,82
4,ACAD FOR CONSERVATION & ENVIRONMENT,81
5,ACAD-SCHOLARSHIP & ENTREPRENEURSHIP,82
6,ACADEMY FOR EXCELLENCE-LEADERSHIP,81
7,ACADEMY FOR HEALTH CAREERS,83
8,ACADEMY FOR LANGUAGE AND TECHNOLOGY,89
9,ACADEMY FOR SOFTWARE ENGINEERING,87
